In [370]:
import pandas as pd
import json
import ta
import os
import torch
from itertools import combinations
from binance.helpers import round_step_size
from binance.client import Client
from binance.enums import *


from datetime import datetime, timedelta, date, time
from itertools import product
from time import strftime
import scipy.stats
import pandas as pd
import numpy as np
import itertools
import warnings
import os.path
import time
import copy
import json
import os
import ta
import ast


import importlib.util
warnings.filterwarnings("ignore")

In [419]:


class BACKTEST_NEW():
    def __init__(self, INPUT_LIST, START_DATE, LEVERAGE, PERCENTAGE_USED, STARTING_BALANCE, FLAG):
        self.INPUT_LIST         = INPUT_LIST
        self.START_DATE         = START_DATE
        self.LEVERAGE           = LEVERAGE
        self.PERCENTAGE_USED    = PERCENTAGE_USED
        self.STARTING_BALANCE   = STARTING_BALANCE
        self.FLAG = FLAG
        self.API_DATA           = self.BACKTEST()

    def BACKTEST(self):
        DF_UPPER                    = self.INPUT_LIST[1]
        DF_LOWER                    = self.INPUT_LIST[0]
        self.START_DATE             = pd.to_datetime(self.START_DATE)
        DF_UPPER                    = DF_UPPER[DF_UPPER['DATE'] >= self.START_DATE].reset_index(drop=True)
        DF_UPPER['STR']             = (DF_UPPER['DATE'].dt.date).astype("string") + ' ' + (DF_UPPER['DATE'].dt.time).astype("string")
        DF_LOWER['STR']             = (DF_LOWER['DATE'].dt.date).astype("string") + ' ' + (DF_LOWER['DATE'].dt.time).astype("string")
        DATA_MAIN                   = DF_UPPER[['STR', 'SIGNAL', 'TP', 'SL']]
        FINAL_DF                    = DF_LOWER.merge(DATA_MAIN, how='left', on=['STR'])
        FINAL_DF['SIGNAL']          = FINAL_DF['SIGNAL'].fillna('STATIC')
        FINAL_DF['TRADE_STATUS']    = None 

        TRADES_TP_SL                = self.RUN_BACKTEST(FINAL_DF)
        TRADES_NEXT_TRADE           = self.EXIT_AT_NEXT_TRADE(FINAL_DF)
        TRADES_COMBINED             = self.COMBINED(TRADES_TP_SL, TRADES_NEXT_TRADE)
        SCENARIOS                   = ['TP_SL_ONLY', 'LEADING_ONLY', 'TP_SL_LEADING']
        SCENARIOS_VARIABLES         = [TRADES_TP_SL, TRADES_NEXT_TRADE, TRADES_COMBINED]
        TIME_LIST                   = []

        for A in range(len(SCENARIOS)): 
            if A == 0:              

                SCENARIOS_VARIABLES[A]  = self.REMOVE_OVERLAPPING_TRADES(SCENARIOS_VARIABLES[A])
                PORTFOLIO_RESULTS       = self.PORTFOLIO_TIMELINE(SCENARIOS_VARIABLES[A])
                OVERVIEW                = self.OVERVIEW(SCENARIOS_VARIABLES[A], SCENARIOS[A])
                OVERVIEW_DF             = OVERVIEW.copy()

            else:
            
                PORTFOLIO_RESULTS       = self.PORTFOLIO_TIMELINE(SCENARIOS_VARIABLES[A])
                OVERVIEW                = self.OVERVIEW(SCENARIOS_VARIABLES[A], SCENARIOS[A])             
                OVERVIEW_DF             = pd.concat([OVERVIEW_DF, OVERVIEW], ignore_index=True)


            TIME_LIST.append(PORTFOLIO_RESULTS)

        return OVERVIEW_DF  #[OVERVIEW_DF, TIME_LIST, SCENARIOS_VARIABLES, FINAL_DF]


    def COMBINED(self, TRADES_TP_SL, TRADES_NEXT_TRADE):

        if TRADES_TP_SL.empty: 
            TRADES_NEXT_TRADE['START_INDEX'] = ''                 
            return TRADES_NEXT_TRADE[['START_INDEX', 'END_INDEX', 'STATUS', 'TIME_COMPLETED', 'PERCENTAGE_CHANGE']]
        TRADES_COMBINED                         = TRADES_TP_SL.merge(TRADES_NEXT_TRADE, on="START_INDEX", how="inner")
        TRADES_COMBINED['END_INDEX']            = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['END_INDEX_x'], TRADES_COMBINED['END_INDEX_y'])
        TRADES_COMBINED['STATUS']               = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['STATUS_x'], TRADES_COMBINED['STATUS_y'])   
        TRADES_COMBINED['TIME_COMPLETED']       = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['TIME_COMPLETED_x'], TRADES_COMBINED['TIME_COMPLETED_y'])   
        TRADES_COMBINED['PERCENTAGE_CHANGE']    = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['PERCENTAGE_CHANGE_x'], TRADES_COMBINED['PERCENTAGE_CHANGE_y'])  
        TRADES_COMBINED                         = TRADES_COMBINED[['START_INDEX', 'END_INDEX', 'STATUS', 'TIME_COMPLETED', 'PERCENTAGE_CHANGE']]

        return TRADES_COMBINED


    def RUN_BACKTEST(self, FINAL_DF):
        RESULTS, ACTIVE_TRADE = [], None

        for IDX, ROW in FINAL_DF.iterrows():
            if ACTIVE_TRADE is None and ROW['SIGNAL'] != "STATIC":
                ACTIVE_TRADE = {"INDEX": IDX, "TRADE": ROW}

            if ACTIVE_TRADE:
                OUTCOME, COMPLETED_TIME, COMPLETED_INDEX, PERCENTAGE_CHANGE = self.TRADE_ANALYSIS(ACTIVE_TRADE["TRADE"], FINAL_DF.iloc[IDX:])

                if OUTCOME in ["SUCCESS", "FAILURE"]:
                    RESULTS.append({"START_INDEX"       : ACTIVE_TRADE["INDEX"],
                                    "END_INDEX"         : COMPLETED_INDEX,
                                    "STATUS"            : OUTCOME,
                                    "TIME_COMPLETED"    : COMPLETED_TIME,
                                    "PERCENTAGE_CHANGE" : PERCENTAGE_CHANGE})

                    FINAL_DF.at[ACTIVE_TRADE["INDEX"], 'TRADE_STATUS'] = OUTCOME
                    ACTIVE_TRADE = None  

        return pd.DataFrame(RESULTS)




    def TRADE_ANALYSIS(self, ROW, TRADE_DATA):
        DIRECTION, TRADE_START, TRADE_OPEN      = ROW['SIGNAL'], ROW['DATE'], ROW['OPEN']
        TRADE_TP, TRADE_SL                      = ROW['TP'], ROW['SL']
        if TRADE_DATA.empty:                    return "FAILURE", TRADE_START, TRADE_DATA.index[-1], 0

        if DIRECTION == "LONG":
            TP_IDX = TRADE_DATA.loc[TRADE_DATA['HIGH'] >= TRADE_TP, 'DATE'].first_valid_index()
            SL_IDX = TRADE_DATA.loc[TRADE_DATA['LOW'] <= TRADE_SL, 'DATE'].first_valid_index()
        else:  
            TP_IDX = TRADE_DATA.loc[TRADE_DATA['LOW'] <= TRADE_TP, 'DATE'].first_valid_index()
            SL_IDX = TRADE_DATA.loc[TRADE_DATA['HIGH'] >= TRADE_SL, 'DATE'].first_valid_index()

        TP_HIT = TRADE_DATA.at[TP_IDX, 'DATE'] if TP_IDX in TRADE_DATA.index else None
        SL_HIT = TRADE_DATA.at[SL_IDX, 'DATE'] if SL_IDX in TRADE_DATA.index else None

        if TP_IDX is not None and SL_IDX is not None:       return ("SUCCESS", TP_HIT, TP_IDX, abs((TRADE_TP - TRADE_OPEN) / TRADE_OPEN) * 100) if TP_IDX < SL_IDX else ("FAILURE", SL_HIT, SL_IDX, -abs((TRADE_SL - TRADE_OPEN) / TRADE_OPEN) * 100)
        elif TP_IDX is not None:                            return "SUCCESS", TP_HIT, TP_IDX, abs((TRADE_TP - TRADE_OPEN) / TRADE_OPEN) * 100
        elif SL_IDX is not None:                            return "FAILURE", SL_HIT, SL_IDX, -abs((TRADE_SL - TRADE_OPEN) / TRADE_OPEN) * 100

        return "FAILURE", TRADE_DATA['DATE'].iloc[-1], TRADE_DATA.index[-1], 0



    def EXIT_AT_NEXT_TRADE(self, FINAL_DF):
        if self.FLAG == 'YES':  print(FINAL_DF)

        TRADE_DF                = (FINAL_DF[FINAL_DF['SIGNAL'] != "STATIC"])

        if TRADE_DF.empty:      return pd.DataFrame(columns=["END_INDEX", "STATUS", "TIME_COMPLETED", "PERCENTAGE_CHANGE"])
    
        TRADE_DF["END_INDEX"]   = TRADE_DF.index.to_series().shift(-1)
        TRADE_DF["EXIT_DATE"]   = TRADE_DF["DATE"].shift(-1)
        TRADE_DF["EXIT_PRICE"]  = TRADE_DF["OPEN"].shift(-1)

        if self.FLAG == 'YES':  print(TRADE_DF)

        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("END_INDEX")]   = FINAL_DF.index[-1]
        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("EXIT_DATE")]    = FINAL_DF["DATE"].iloc[-1]
        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("EXIT_PRICE")]   = FINAL_DF["CLOSE"].iloc[-1]

        TRADE_DF["STATUS"]              = np.where(((TRADE_DF["EXIT_PRICE"] > TRADE_DF["OPEN"]) & (TRADE_DF["SIGNAL"] == "LONG")) | ((TRADE_DF["EXIT_PRICE"] < TRADE_DF["OPEN"]) & (TRADE_DF["SIGNAL"] == "SHORT")),  "SUCCESS", "FAILURE")
        TRADE_DF["PERCENTAGE_CHANGE"]   = np.where(TRADE_DF["SIGNAL"] == "LONG", ((TRADE_DF["EXIT_PRICE"] - TRADE_DF["OPEN"]) / TRADE_DF["OPEN"]) * 100, ((TRADE_DF["OPEN"] - TRADE_DF["EXIT_PRICE"]) / TRADE_DF["OPEN"]) * 100)

        return TRADE_DF[["END_INDEX", "STATUS", "EXIT_DATE", "PERCENTAGE_CHANGE"]].reset_index().rename(columns={"index": "START_INDEX", "EXIT_DATE": "TIME_COMPLETED"})




    def PORTFOLIO_TIMELINE(self, INPUT_DF):


        BALANCE, N                      = self.STARTING_BALANCE, len(INPUT_DF)
        FEE_RATE, ALLOCATION            = 0.00075, 0.985 if self.PERCENTAGE_USED == 1 else self.PERCENTAGE_USED
        if INPUT_DF.empty:              return pd.DataFrame(columns=["START_AMOUNT", "TRADE_AMOUNT", "LEVERED_AMOUNT", "PERCENTAGE_CHANGE", "GROSS_PROFIT", "FEES", "NET_PROFIT", "END_AMOUNT"])

        start_amounts                   = np.zeros(N)
        trade_amounts                   = np.zeros(N)
        levered_amounts                 = np.zeros(N)
        gross_profits                   = np.zeros(N)
        fees                            = np.zeros(N)
        net_profits                     = np.zeros(N)
        end_amounts                     = np.zeros(N)
        start_amounts[0]                = BALANCE 
        percentage_changes              = INPUT_DF["PERCENTAGE_CHANGE"].values / 100  
        zero_balance_index              = None  

        for i in range(N):
            if zero_balance_index is not None: start_amounts[i] = trade_amounts[i] = levered_amounts[i] = gross_profits[i] = fees[i] = net_profits[i] = end_amounts[i] = 0
            else:
                trade_amounts[i]        = start_amounts[i] * ALLOCATION
                levered_amounts[i]      = trade_amounts[i] * self.LEVERAGE
                gross_profits[i]        = levered_amounts[i] * percentage_changes[i]
                fees[i]                 = levered_amounts[i] * FEE_RATE
                net_profits[i]          = gross_profits[i] - fees[i]
                end_amounts[i]          = start_amounts[i] + net_profits[i]

                if end_amounts[i] <= 0:
                    zero_balance_index  = i 
                    end_amounts[i]      = 0  

            if i < N - 1 and zero_balance_index is None:
                start_amounts[i + 1]    = end_amounts[i]

        INPUT_DF["START_AMOUNT"]        = start_amounts
        INPUT_DF["TRADE_AMOUNT"]        = trade_amounts
        INPUT_DF["LEVERED_AMOUNT"]      = levered_amounts
        INPUT_DF["GROSS_PROFIT"]        = gross_profits
        INPUT_DF["FEES"]                = fees
        INPUT_DF["NET_PROFIT"]          = net_profits
        INPUT_DF["END_AMOUNT"]          = end_amounts

        return INPUT_DF[["START_AMOUNT", "TRADE_AMOUNT", "LEVERED_AMOUNT", "PERCENTAGE_CHANGE", "GROSS_PROFIT", "FEES", "NET_PROFIT", "END_AMOUNT"]]




    def OVERVIEW(self, RESULTS_TABLE, SCENARIO_DESC):

        if RESULTS_TABLE.empty:     return pd.DataFrame([{"METHOD": SCENARIO_DESC, "END_BALANCE": self.STARTING_BALANCE, "WIN_RATE": 0, "NBR_TRADES": 0}])
        WIN_TRADES                  = (RESULTS_TABLE["PERCENTAGE_CHANGE"] > 0).sum()
        TOTAL_TRADES                = len(RESULTS_TABLE)
        WIN_RATE                    = WIN_TRADES / TOTAL_TRADES if TOTAL_TRADES > 0 else 0

        return pd.DataFrame([{  "METHOD"        : SCENARIO_DESC,
                                "END_BALANCE"   : RESULTS_TABLE["END_AMOUNT"].iloc[-1],
                                "WIN_RATE"      : WIN_RATE,
                                "NBR_TRADES"    : TOTAL_TRADES}])
    


    def REMOVE_OVERLAPPING_TRADES(self, DF):

        DF_USE = DF.copy()
        
        if len(DF_USE) >0:

            DF_USE = DF_USE.sort_values("START_INDEX").reset_index(drop=True)
            NON_OVERLAPPING = []  

            for i in range(len(DF_USE)):
                if i == 0:          NON_OVERLAPPING.append(DF_USE.iloc[i]) 
                else:
                    PREV_TRADE      = NON_OVERLAPPING[-1]
                    CURRENT_TRADE   = DF_USE.iloc[i]

                    if PREV_TRADE["END_INDEX"] > CURRENT_TRADE["START_INDEX"]:     continue  
                    else:    NON_OVERLAPPING.append(CURRENT_TRADE)

            return pd.DataFrame(NON_OVERLAPPING)

        else: return DF_USE







In [385]:
def DATA_CLEANING(BASE_DATA, REVIEW_PERIOD, HISTORY_CONTRIBUTING, TIME):

    BASE_DATA["DATE"]               = pd.to_datetime(BASE_DATA["DATE"], dayfirst=True)
    BASE_DATA                       = BASE_DATA[['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]

    if TIME == '4H':                MULTIPLE = 6
    else:                           MULTIPLE = 1

    LIST_OF_DATA                    = []
    STARTING_POINT                  = (len(BASE_DATA) - ((REVIEW_PERIOD + HISTORY_CONTRIBUTING) * MULTIPLE)) if (len(BASE_DATA) - ((REVIEW_PERIOD + HISTORY_CONTRIBUTING) * MULTIPLE)) > 0 else 0
    VALIDATION_POINT                = len(BASE_DATA) - ((REVIEW_PERIOD) * MULTIPLE)

    FULL_SCOPE                      = BASE_DATA.iloc[STARTING_POINT:].reset_index(drop=True) 
    VALID_SCOPE                     = BASE_DATA.iloc[VALIDATION_POINT:].reset_index(drop=True) 

    for B in range((REVIEW_PERIOD * MULTIPLE)):

        TRAINING_DATA               = FULL_SCOPE.copy()
        END                         = B - (REVIEW_PERIOD * MULTIPLE)
        TRAIN_DF                    = TRAINING_DATA.iloc[:END].reset_index(drop=True) 

        LIST_OF_DATA.append(TRAIN_DF)

    return LIST_OF_DATA, VALID_SCOPE









#2️⃣ Short-Term vs Long-Term Predictions
#	•	Short-Term Prediction (1-4 time steps ahead) → Lookback of 16 to 50 candles works well.
#	•	Medium-Term (5-50 steps ahead) → Use 50 to 100 candles.
#	•	Long-Term (100+ steps ahead) → Lookback should be 100-200, but model accuracy declines.


def PREDICTION_LOGIC(PIPELINE, LIST_OF_DATA, REVIEW_PERIOD, TIME, LOOKBACK=20):
    if TIME == '4H':                MULTIPLE = 6
    else:                           MULTIPLE = 1

    PREDICTIONS, TP_AI_2            = [], []
    SL_AI_LOW, SL_AI_MAX            = [], []

    for B in range((REVIEW_PERIOD * MULTIPLE)):
        print(f'{B} OF {REVIEW_PERIOD * MULTIPLE} COMPLETE')

        CONTEXT                     = LIST_OF_DATA[B]['CLOSE'].values[(-1*(LOOKBACK+1)):] 
        CONTEXT_TENSOR              = torch.tensor([np.array(CONTEXT, dtype=np.float32)])  
        FORECAST                    = PIPELINE.predict(CONTEXT_TENSOR, 1)  
        LOW, MEDIAN, HIGH           = np.quantile(FORECAST[0].squeeze().numpy(), [0.1, 0.5, 0.9], axis=0)

        PREDICTIONS.append([LOW, MEDIAN, HIGH])

        AI_CON                      = LIST_OF_DATA[B][['OPEN', 'HIGH', 'LOW', 'CLOSE']].values.tolist()
        CONTEXT_                    = AI_CON[-50:]
        CONTEXT_TENSOR_             = torch.tensor(np.array(CONTEXT_, dtype=np.float32))
        FORECAST_                   = PIPELINE.predict(CONTEXT_TENSOR_, 4)
        LOW_, MEDIAN_, HIGH_        = np.quantile(FORECAST_[0].numpy(), [0.1, 0.5, 0.9], axis=0)

        TP_AI_2.append(np.quantile(FORECAST_[0][:, 3].numpy(), 0.5))
        SL_AI_LOW.append(np.min(FORECAST_[0][:, 2].numpy()))
        SL_AI_MAX.append(np.max(FORECAST_[0][:, 1].numpy()))

    return PREDICTIONS, TP_AI_2, SL_AI_LOW, SL_AI_MAX







def FINAL_PROCESS_LOOP(PREDICTIONS, TP_AI_2, SL_AI_LOW, SL_AI_MAX, VALID_DF):

    VALID_DF["TP_AI_1"]             = [float(PRED[1]) for PRED in PREDICTIONS]
    VALID_DF["SIGNAL_1"]              = np.where(VALID_DF["OPEN"] > VALID_DF["TP_AI_1"], 'SHORT', 'LONG')
    VALID_DF["STATUS_1"]              = np.where(VALID_DF["SIGNAL_1"]=='LONG', (np.where(VALID_DF["TP_AI_1"]<=VALID_DF["HIGH"], 'SUCCESS', 'FAILURE')) , (np.where(VALID_DF["TP_AI_1"]>=VALID_DF["LOW"],  'SUCCESS', 'FAILURE')))
    VALID_DF["P_GAIN_1"]              = np.where(VALID_DF["SIGNAL_1"]=='LONG', round(((VALID_DF["TP_AI_1"] / VALID_DF["OPEN"]) - 1)*100,2) , round((1- (VALID_DF["TP_AI_1"] / VALID_DF["OPEN"]))*100,2))

    VALID_DF["TP_AI_2"]             = [float(TP_AI) for TP_AI in TP_AI_2]
    VALID_DF["SIGNAL_2"]              = np.where(VALID_DF["OPEN"] > VALID_DF["TP_AI_2"], 'SHORT', 'LONG')
    VALID_DF["STATUS_2"]              = np.where(VALID_DF["SIGNAL_2"]=='LONG', (np.where(VALID_DF["TP_AI_2"]<=VALID_DF["HIGH"], 'SUCCESS', 'FAILURE')), (np.where(VALID_DF["TP_AI_2"]>=VALID_DF["LOW"],  'SUCCESS', 'FAILURE')))
    VALID_DF["P_GAIN_2"]              = np.where(VALID_DF["SIGNAL_2"]=='LONG', round(((VALID_DF["TP_AI_2"] / VALID_DF["OPEN"]) - 1)*100,2) , round((1- (VALID_DF["TP_AI_2"] / VALID_DF["OPEN"]))*100,2))

    VALID_DF["SL_AI_11"]             = [float(S_A_1) for S_A_1 in SL_AI_LOW]
    VALID_DF["SL_AI_22"]             = [float(S_A_2) for S_A_2 in SL_AI_MAX]
    VALID_DF["SL_AI_1"]               = np.where(VALID_DF["SIGNAL_1"]=='LONG', np.where(((VALID_DF["SL_AI_11"]/VALID_DF["OPEN"] < 0.95) | (VALID_DF["SL_AI_11"]/VALID_DF["OPEN"] > 1)), VALID_DF["OPEN"] * 0.95, VALID_DF["SL_AI_11"]), np.where(((VALID_DF["SL_AI_22"]/VALID_DF["OPEN"] > 1.05) | (VALID_DF["SL_AI_22"]/VALID_DF["OPEN"] < 1)), VALID_DF["OPEN"] * 1.05, VALID_DF["SL_AI_22"]))
    VALID_DF["SL_AI_2"]               = np.where(VALID_DF["SIGNAL_2"]=='LONG', np.where(((VALID_DF["SL_AI_11"]/VALID_DF["OPEN"] < 0.95) | (VALID_DF["SL_AI_11"]/VALID_DF["OPEN"] > 1)), VALID_DF["OPEN"] * 0.95, VALID_DF["SL_AI_11"]), np.where(((VALID_DF["SL_AI_22"]/VALID_DF["OPEN"] > 1.05) | (VALID_DF["SL_AI_22"]/VALID_DF["OPEN"] < 1)), VALID_DF["OPEN"] * 1.05, VALID_DF["SL_AI_22"]))
    
    del VALID_DF["SL_AI_11"]
    del VALID_DF["SL_AI_22"]

    return VALID_DF






def ATR_STOP_LOSS(DF, PERIOD=14, RISK_MULTIPLIER=2.0):

    DF['High-Low']      = DF['HIGH'] - DF['LOW']
    DF['High-Close']    = abs(DF['HIGH'] - DF['CLOSE'].shift(1))
    DF['Low-Close']     = abs(DF['LOW'] - DF['CLOSE'].shift(1))
    DF['True_Range']    = DF[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    DF['ATR']           = DF['True_Range'].rolling(window=PERIOD).mean()
    DF['ATR_SL_PREV_1'] = np.where(DF['SIGNAL_1'].shift(-1) == 'LONG',DF['CLOSE'] - (DF['ATR'] * RISK_MULTIPLIER),DF['CLOSE'] + (DF['ATR'] * RISK_MULTIPLIER))
    DF['ATR_SL_PREV_2'] = np.where(DF['SIGNAL_2'].shift(-1) == 'LONG',DF['CLOSE'] - (DF['ATR'] * RISK_MULTIPLIER),DF['CLOSE'] + (DF['ATR'] * RISK_MULTIPLIER))
    DF['SL_ATR_1']      = DF['ATR_SL_PREV_1'].shift(1)
    DF['SL_ATR_2']      = DF['ATR_SL_PREV_2'].shift(1)
    DEL_COLS            = ['ATR_SL_PREV_1', 'ATR_SL_PREV_2', 'ATR', 'True_Range', 'Low-Close', 'High-Close', 'High-Low']

    for COLS in DEL_COLS: del DF[COLS]
    return DF






In [373]:



class EXPORT_RAW_DATA():

    def __init__(self, DF, FILEPATH):
        self.DF                     = DF
        self.FILEPATH               = FILEPATH
        self.DF                     = self.READ_IN_RAW_DATA()

    def READ_IN_RAW_DATA(self):

        self.DF.to_csv(self.FILEPATH, index=False)
        print(f"Data has been exported to - {self.FILEPATH}")

        return self.DF



class READ_IN_DATA():

    def __init__(self, COINS, TIMEFRAME, PATH):
        self.PATH                               = PATH
        self.COINS                              = COINS
        self.TIMEFRAME                          = TIMEFRAME
        self.LIST                               = self.READ_IN_RAW_DATA()


    def READ_IN_RAW_DATA(self):
        COIN_HIERARCHY = []

        for a in range(len(self.COINS)):
            TIME_FRAME_HIERARCHY = []
            for b in range(len(self.TIMEFRAME)):

                
                FILENAME                    = self.COINS[a] + ' -- ' + self.TIMEFRAME[b] + '.csv'
                FILE_PATH                   = self.PATH + FILENAME
                DF                          = (pd.read_csv(FILE_PATH)).iloc[:-1]
                DF["DATE"]                  = pd.to_datetime(DF["DATE"])

                TIME_FRAME_HIERARCHY.append(DF)

            COIN_HIERARCHY.append(TIME_FRAME_HIERARCHY)

        return COIN_HIERARCHY



class READ_IN_DATA_AI():

    def __init__(self, COINS, TIMEFRAME, PIPE, PATH):
        self.PATH                               = PATH
        self.PIPE                               = PIPE
        self.COINS                              = COINS
        self.TIMEFRAME                          = TIMEFRAME
        self.LIST                               = self.READ_IN_RAW_DATA()


    def READ_IN_RAW_DATA(self):
        COIN_HIERARCHY = []

        for a in range(len(self.COINS)):
            TIME_FRAME_HIERARCHY = []
            for b in range(len(self.TIMEFRAME)):

                FILENAME                    = f'{self.COINS[a]} -- {self.TIMEFRAME[b]} -- {self.PIPE} -- AI TEST.csv'
                FILE_PATH                   = self.PATH + FILENAME
                DF                          = (pd.read_csv(FILE_PATH)).iloc[:-1]
                DF["DATE"]                  = pd.to_datetime(DF["DATE"])

                TIME_FRAME_HIERARCHY.append(DF)

            COIN_HIERARCHY.append(TIME_FRAME_HIERARCHY)

        return COIN_HIERARCHY













In [401]:






TIMEFRAME_RDS                               = ['4H']
COINS                                       = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
COINS                                       = ['XRPUSDT']
COINS                                       = ['BTCUSDT', 'ETHUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']

PATH_RDS                                    = r'/Users/westhomas/Desktop/ENKI/ENKI_AI_DEVELOPMENT/__AI__DATA__/_DATA_/'
PATH_RDF                                    = r'/Users/westhomas/Desktop/ENKI/ENKI_AI_DEVELOPMENT/__AI__DATA__/_FINAL_VIEW_/'

DATA_RDS                                    = READ_IN_DATA(COINS, TIMEFRAME_RDS, PATH_RDS)
DATA_RDC                                    = READ_IN_DATA(COINS, TIMEFRAME_RDS, PATH_RDS)


FEATURE_LIST                                = [ "DATE", "EMA_4", "EMA_8", "EMA_12", "EMA_24", "EMA_50", "EMA_100", "EMA_200",
                                                "RSI_14", "RSI_30", "MACD", "MACD_SIGNAL", "BB_UPPER", "BB_LOWER", 
                                                "VWAP", "ROC", "%K", "%D", "AVG_VOLATILITY_7", "AVG_VOLATILITY_10", "AVG_VOLATILITY_20"]



HISTORY_CONTRIBUTING                        = 1000
REVIEW_PERIOD                               = 700


PIPE_TOOL = ["amazon/chronos-t5-large"]     #, 
PIPE_DESC = ["LARGE_MODEL"]                 #, "LARGE_MODEL"



for a in range(len(PIPE_TOOL)):
    PIPELINE = ChronosPipeline.from_pretrained(PIPE_TOOL[a], device_map = "mps",  torch_dtype = torch.float32)

    VALID_LIST = []
    for x in range(len(COINS)):

        COMPONENT_LIST = []
        for y in range(len(TIMEFRAME_RDS)):

            FILENAME                                    = f'{COINS[x]} -- {TIMEFRAME_RDS[y]} -- {PIPE_DESC[a]} -- AI TEST.csv'
            print(FILENAME)

            LIST_OF_DATA, VALIDATION_DATA               = DATA_CLEANING(DATA_RDS.LIST[x][y], REVIEW_PERIOD, HISTORY_CONTRIBUTING, TIMEFRAME_RDS[y])
            PREDICTIONS, TP_AI_2, SL_AI_LOW, SL_AI_MAX  = PREDICTION_LOGIC(PIPELINE, LIST_OF_DATA, REVIEW_PERIOD, TIMEFRAME_RDS[y])
            VALIDATION_DATA                             = FINAL_PROCESS_LOOP(PREDICTIONS, TP_AI_2, SL_AI_LOW, SL_AI_MAX, VALIDATION_DATA)
            VALIDATION_DATA                             = ATR_STOP_LOSS(VALIDATION_DATA, PERIOD=14, RISK_MULTIPLIER=2.0)

            DATA_RDC.LIST[x][y]                         = DATA_RDC.LIST[x][y][FEATURE_LIST]
            MERGED_DF                                   = VALIDATION_DATA.merge(DATA_RDC.LIST[x][y], on="DATE", how="inner")
            MERGED_DF                                   = MERGED_DF[['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TP_AI_1', 'TP_AI_2', 'SL_AI_1', 'SL_AI_2', 'SL_ATR_1', 'SL_ATR_2',
                                                                    'SIGNAL_1', 'STATUS_1', 'P_GAIN_1', 'SIGNAL_2', 'STATUS_2', 'P_GAIN_2', 
                                                                    "EMA_4", "EMA_8", "EMA_12", "EMA_24", "EMA_50", "EMA_100", "EMA_200",
                                                                    "RSI_14", "RSI_30", "MACD", "MACD_SIGNAL", "BB_UPPER", "BB_LOWER", 
                                                                    "ROC", "%K", "%D", "AVG_VOLATILITY_7", "AVG_VOLATILITY_10", "AVG_VOLATILITY_20"]]
            
            MERGED_DF                                   = MERGED_DF[15:].reset_index(drop=True)
            DF_OUT                                      = EXPORT_RAW_DATA(MERGED_DF, (PATH_RDF + FILENAME))

            COMPONENT_LIST.append(VALIDATION_DATA)
        VALID_LIST.append(COMPONENT_LIST)









BTCUSDT -- 4H -- LARGE_MODEL -- AI TEST.csv
0 OF 4200 COMPLETE
1 OF 4200 COMPLETE
2 OF 4200 COMPLETE
3 OF 4200 COMPLETE
4 OF 4200 COMPLETE
5 OF 4200 COMPLETE
6 OF 4200 COMPLETE
7 OF 4200 COMPLETE
8 OF 4200 COMPLETE
9 OF 4200 COMPLETE
10 OF 4200 COMPLETE
11 OF 4200 COMPLETE
12 OF 4200 COMPLETE
13 OF 4200 COMPLETE
14 OF 4200 COMPLETE
15 OF 4200 COMPLETE
16 OF 4200 COMPLETE
17 OF 4200 COMPLETE
18 OF 4200 COMPLETE
19 OF 4200 COMPLETE
20 OF 4200 COMPLETE
21 OF 4200 COMPLETE
22 OF 4200 COMPLETE
23 OF 4200 COMPLETE
24 OF 4200 COMPLETE
25 OF 4200 COMPLETE
26 OF 4200 COMPLETE
27 OF 4200 COMPLETE
28 OF 4200 COMPLETE
29 OF 4200 COMPLETE
30 OF 4200 COMPLETE
31 OF 4200 COMPLETE
32 OF 4200 COMPLETE
33 OF 4200 COMPLETE
34 OF 4200 COMPLETE
35 OF 4200 COMPLETE
36 OF 4200 COMPLETE
37 OF 4200 COMPLETE
38 OF 4200 COMPLETE
39 OF 4200 COMPLETE
40 OF 4200 COMPLETE
41 OF 4200 COMPLETE
42 OF 4200 COMPLETE
43 OF 4200 COMPLETE
44 OF 4200 COMPLETE
45 OF 4200 COMPLETE
46 OF 4200 COMPLETE
47 OF 4200 COMPLETE
48 OF 

In [383]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [336]:

class READ_IN_DATA_LOWER():

    def __init__(self, COINS, TIMEFRAME, PATH):
        self.PATH                               = PATH
        self.COINS                              = COINS
        self.TIMEFRAME                          = TIMEFRAME
        self.LIST                               = self.READ_IN_RAW_DATA()


    def READ_IN_RAW_DATA(self):
        COIN_HIERARCHY = []

        for a in range(len(self.COINS)):
            TIME_FRAME_HIERARCHY = []
            for b in range(len(self.TIMEFRAME)):

                
                FILENAME                    = self.COINS[a] + ' -- ' + self.TIMEFRAME[b] + '.csv'
                FILE_PATH                   = self.PATH + FILENAME
                DF                          = (pd.read_csv(FILE_PATH)).iloc[:-1]
                DF["DATE"]                  = pd.to_datetime(DF["DATE"], format="%d/%m/%Y %H:%M", errors="coerce")
                DF                          = DF[['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']]

                TIME_FRAME_HIERARCHY.append(DF)

            COIN_HIERARCHY.append(TIME_FRAME_HIERARCHY)

        return COIN_HIERARCHY




In [355]:


def PROCESSING(DF, TP_, MIN_THRESHOLD, MAX_THRESHOLD, INDICATORS):

    if TP_[0] == 'TP_AI_1':                         GRADE = 1
    else:                                           GRADE = 2

    DF['SIGNAL']                                    = np.where(((DF[f'P_GAIN_{GRADE}']>=MIN_THRESHOLD) & (DF[f'P_GAIN_{GRADE}']<=MAX_THRESHOLD)), DF[f'SIGNAL_{GRADE}'], "STATIC")
    DF['TP']                                        = DF[TP_[0]]
    DF['SL']                                        = DF[TP_[1]]

    if INDICATORS == 'ALL':                         pass
    elif INDICATORS == 'STOCH_10':                  DF['SIGNAL'] = np.where(((DF['%K'] > 10) & (DF['%D'] > 10) & (DF['%K'] < 90) & (DF['%D'] < 90)), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'STOCH_20':                  DF['SIGNAL'] = np.where(((DF['%K'] > 20) & (DF['%D'] > 20) & (DF['%K'] < 80) & (DF['%D'] < 80)), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'STOCH_30':                  DF['SIGNAL'] = np.where(((DF['%K'] > 30) & (DF['%D'] > 30) & (DF['%K'] < 70) & (DF['%D'] < 70)), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'STOCH_40':                  DF['SIGNAL'] = np.where(((DF['%K'] > 40) & (DF['%D'] > 40) & (DF['%K'] < 60) & (DF['%D'] < 60)), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'AVG_VOL_P_GAIN_1_1':        DF['SIGNAL'] = np.where((DF['AVG_VOLATILITY_7'] > (1 * DF[f'P_GAIN_{GRADE}'])), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'AVG_VOL_P_GAIN_2_1':        DF['SIGNAL'] = np.where((DF['AVG_VOLATILITY_7'] > (2 * DF[f'P_GAIN_{GRADE}'])), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'AVG_VOL_P_GAIN_3_1':        DF['SIGNAL'] = np.where((DF['AVG_VOLATILITY_7'] > (3 * DF[f'P_GAIN_{GRADE}'])), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'AVG_VOL_P_GAIN_1_2':        DF['SIGNAL'] = np.where((DF['AVG_VOLATILITY_20'] > (1 * DF[f'P_GAIN_{GRADE}'])), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'AVG_VOL_P_GAIN_2_2':        DF['SIGNAL'] = np.where((DF['AVG_VOLATILITY_20'] > (2 * DF[f'P_GAIN_{GRADE}'])), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'AVG_VOL_P_GAIN_3_2':        DF['SIGNAL'] = np.where((DF['AVG_VOLATILITY_20'] > (3 * DF[f'P_GAIN_{GRADE}'])), DF['SIGNAL'], 'STATIC')
    elif INDICATORS == 'ROC_1':                     DF['SIGNAL'] = np.where((abs(DF['ROC']) > 1), np.where(((DF['ROC'] > 0) & (DF['SIGNAL'] == "LONG")), DF['SIGNAL'], 'STATIC'), np.where(((DF['ROC'] < 0) & (DF['SIGNAL'] == "SHORT")), DF['SIGNAL'], 'STATIC'))
    elif INDICATORS == 'ROC_2':                     DF['SIGNAL'] = np.where((abs(DF['ROC']) > 2), np.where(((DF['ROC'] > 0) & (DF['SIGNAL'] == "LONG")), DF['SIGNAL'], 'STATIC'), np.where(((DF['ROC'] < 0) & (DF['SIGNAL'] == "SHORT")), DF['SIGNAL'], 'STATIC'))
    elif INDICATORS == 'ROC_3':                     DF['SIGNAL'] = np.where((abs(DF['ROC']) > 3), np.where(((DF['ROC'] > 0) & (DF['SIGNAL'] == "LONG")), DF['SIGNAL'], 'STATIC'), np.where(((DF['ROC'] < 0) & (DF['SIGNAL'] == "SHORT")), DF['SIGNAL'], 'STATIC'))


    DF = DF[['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'SIGNAL', 'TP', 'SL']]

    return DF





In [420]:


BACKTEST_RESULTS_DF                             = pd.DataFrame(columns=["FILE", "STRATEGY", "TP_SL", "END_BALANCE__1", "END_BALANCE__2", "END_BALANCE__3", "WIN_RATE__1", "WIN_RATE__2", "WIN_RATE__3", "NBR_TRADES__1", "NBR_TRADES__2", "NBR_TRADES__3", "MAX_END_BALANCE"])
SCENARIOS_INDICATORS                            = ['ALL', 'STOCH_10', 'STOCH_20', 'STOCH_30', 'STOCH_40', 'ROC_1', 'ROC_2', 'ROC_3', 'AVG_VOL_P_GAIN_1_1', 'AVG_VOL_P_GAIN_2_1', 'AVG_VOL_P_GAIN_3_1', 'AVG_VOL_P_GAIN_1_2', 'AVG_VOL_P_GAIN_2_2', 'AVG_VOL_P_GAIN_3_2']
TP_SCENARIOS                                    = [['TP_AI_1', 'SL_AI_1'],['TP_AI_2', 'SL_AI_1'],['TP_AI_1', 'SL_ATR_1'],['TP_AI_2', 'SL_ATR_2']]
SCENARIOS_BASIS                                 = [0.5, 0.75, 1]
SCENARIOS_UPPER                                 = [3, 4, 5, 10]
PIPE_DESC                                       = ["SMALL_MODEL", "LARGE_MODEL"]




TESTING                                         = 'YES'
TEST_TOTAL                                      = len(TP_SCENARIOS) * len(SCENARIOS_BASIS) * len(SCENARIOS_INDICATORS) * len(SCENARIOS_UPPER)


if TESTING == 'YES':

    PATH_RDF                                    = r'/Users/westhomas/Desktop/ENKI/ENKI_AI_DEVELOPMENT/__AI__DATA__/_FINAL_VIEW_/'
    PATH_RDC                                    = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/_RAW_DATA_CLEAN_/'

    TIMEFRAME_RDS                               = ['4H']
    TIMEFRAME_RDC                               = ['4H']
    COINS                                       = ['BTCUSDT', 'ETHUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
    DATA_RDC                                    = READ_IN_DATA_LOWER(COINS, TIMEFRAME_RDC, PATH_RDC)
    PIPE_TOOL = ["amazon/chronos-t5-large"]     #, 
    PIPE_DESC = ["LARGE_MODEL"]                 #, "LARGE_MODEL"



    for a in range(len(PIPE_DESC)):
        DATA                                        = READ_IN_DATA_AI(COINS, TIMEFRAME_RDS, PIPE_DESC[a], PATH_RDF)

        for x in range(len(COINS)):
            for y in range(len(TIMEFRAME_RDS)):
                print(' ')
                COUNTER                             = 0
                FILENAME                            = COINS[x] + ' -- ' + TIMEFRAME_RDS[y] + ' -- ' + PIPE_TOOL[a]

                for TP_ in TP_SCENARIOS:
                    for MIN_THRESHOLD in SCENARIOS_BASIS:
                        for MAX_THRESHOLD in SCENARIOS_UPPER:
                            for INDICATORS in SCENARIOS_INDICATORS:
                                
                                
                                DF_USE              = DATA.LIST[x][0].copy()
                                DF_USE              = PROCESSING(DF_USE, TP_, MIN_THRESHOLD, MAX_THRESHOLD, INDICATORS)
                                DATA_INPUT_LIST     = [DATA_RDC.LIST[x][y].copy(), DF_USE]
                                if COUNTER == 293:  BACKTEST_OUTPUT     = BACKTEST_NEW(DATA_INPUT_LIST, '2022-01-01', 5, 0.5, 100, "YES")
                                else:               BACKTEST_OUTPUT     = BACKTEST_NEW(DATA_INPUT_LIST, '2022-01-01', 5, 0.5, 100, "NO")


                                COUNTER = COUNTER + 1
                                print(f' STRATEGY USED: {COINS[x]} -- {TIMEFRAME_RDS[y]} -- {PIPE_DESC[a]}      STRATEGY NBR : {COUNTER} OF {TEST_TOTAL} TOTAL STRATEGIES. THIS EQUATES TO {round((COUNTER/TEST_TOTAL) * 100, 1)}% COMPLETE')

                                APPEND_DICT                 = { "FILE"              : FILENAME,
                                                                "STRATEGY"          : f'{MIN_THRESHOLD}  --  {MAX_THRESHOLD}  --  {INDICATORS}', 
                                                                "TP_SL"             : f'{TP_[0]}  --  {TP_[1]}', 
                                                                "END_BALANCE__1"    : round(BACKTEST_OUTPUT.API_DATA.at[0, 'END_BALANCE'],2),
                                                                "END_BALANCE__2"    : round(BACKTEST_OUTPUT.API_DATA.at[1, 'END_BALANCE'],2),
                                                                "END_BALANCE__3"    : round(BACKTEST_OUTPUT.API_DATA.at[2, 'END_BALANCE'],2),
                                                                "WIN_RATE__1"       : round(BACKTEST_OUTPUT.API_DATA.at[0, 'WIN_RATE'],2),
                                                                "WIN_RATE__2"       : round(BACKTEST_OUTPUT.API_DATA.at[1, 'WIN_RATE'],2),
                                                                "WIN_RATE__3"       : round(BACKTEST_OUTPUT.API_DATA.at[2, 'WIN_RATE'],2),
                                                                "NBR_TRADES__1"     : round(BACKTEST_OUTPUT.API_DATA.at[0, 'NBR_TRADES'],2),
                                                                "NBR_TRADES__2"     : round(BACKTEST_OUTPUT.API_DATA.at[1, 'NBR_TRADES'],2),
                                                                "NBR_TRADES__3"     : round(BACKTEST_OUTPUT.API_DATA.at[2, 'NBR_TRADES'],2),
                                                                "MAX_END_BALANCE"   : round(BACKTEST_OUTPUT.API_DATA['END_BALANCE'].max(),2)}



                                BACKTEST_RESULTS_DF.loc[len(BACKTEST_RESULTS_DF)] = APPEND_DICT


    
    DF_OUT = EXPORT_RAW_DATA(BACKTEST_RESULTS_DF, (PATH_RDF + 'BACKTEST_OUTPUT_FILE_4H - 3' + '.csv'))








 
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 1 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 0.1% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 2 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 0.3% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 3 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 0.4% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 4 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 0.6% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 5 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 0.7% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 6 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 0.9% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 7 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 1.0% COMPLETE
 STRATEGY USED: BTCUSDT -- 4H -- LARGE_MODEL      STRATEGY NBR : 8 OF 672 TOTAL STRATEGIES. THIS EQUATES TO 1.2% COMPLETE
 STRATEGY USED: BTCUSD

In [404]:
BACKTEST_RESULTS_DF

,FILE,STRATEGY,TP_SL,END_BALANCE__1,END_BALANCE__2,END_BALANCE__3,WIN_RATE__1,WIN_RATE__2,WIN_RATE__3,NBR_TRADES__1,NBR_TRADES__2,NBR_TRADES__3,MAX_END_BALANCE
0,BTCUSDT -- 4H -- amazon/chronos-t5-large,0.5 -- 3 -- ALL,TP_AI_1 -- SL_AI_1,1.12,0.28,2.15,0.79,0.47,0.64,471,799,799,2.15
1,BTCUSDT -- 4H -- amazon/chronos-t5-large,0.5 -- 3 -- STOCH_10,TP_AI_1 -- SL_AI_1,8.27,2.64,5.95,0.82,0.47,0.66,403,626,626,8.27
2,BTCUSDT -- 4H -- amazon/chronos-t5-large,0.5 -- 3 -- STOCH_20,TP_AI_1 -- SL_AI_1,6.60,1.01,4.44,0.78,0.49,0.65,273,403,403,6.60
3,BTCUSDT -- 4H -- amazon/chronos-t5-large,0.5 -- 3 -- STOCH_30,TP_AI_1 -- SL_AI_1,9.96,0.94,11.83,0.75,0.45,0.66,169,224,224,11.83
4,BTCUSDT -- 4H -- amazon/chronos-t5-large,0.5 -- 3 -- STOCH_40,TP_AI_1 -- SL_AI_1,57.50,1.54,57.13,0.79,0.43,0.74,66,74,74,57.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- AVG_VOL_P_GAIN_1_1,TP_AI_2 -- SL_AI_1,0.00,0.00,0.00,0.30,0.49,0.28,140,186,186,0.00
289,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- AVG_VOL_P_GAIN_2_1,TP_AI_2 -- SL_AI_1,57.38,0.00,55.15,0.57,0.67,0.53,14,15,15,57.38
290,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- AVG_VOL_P_GAIN_3_1,TP_AI_2 -- SL_AI_1,89.61,0.00,89.61,0.50,0.00,0.50,2,2,2,89.61
291,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- AVG_VOL_P_GAIN_1_2,TP_AI_2 -- SL_AI_1,0.00,374.89,0.00,0.31,0.53,0.29,138,179,179,374.89


In [405]:


#BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF[BACKTEST_RESULTS_DF['STRATEGY']!='STRATEGY_NBR_246']
#BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF[BACKTEST_RESULTS_DF['END_BALANCE__1']>1000]
#BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF[BACKTEST_RESULTS_DF['END_BALANCE__2']>1000]
BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF.sort_values(by=["MAX_END_BALANCE", "END_BALANCE__2","END_BALANCE__1"], ascending=[False, False, False])
BACKTEST_RESULTS_DF[:50]




,FILE,STRATEGY,TP_SL,END_BALANCE__1,END_BALANCE__2,END_BALANCE__3,WIN_RATE__1,WIN_RATE__2,WIN_RATE__3,NBR_TRADES__1,NBR_TRADES__2,NBR_TRADES__3,MAX_END_BALANCE
117,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- ROC_1,TP_AI_1 -- SL_AI_1,91.17,630.55,99.65,0.80,0.69,0.81,30,32,32,630.55
131,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 4 -- ROC_1,TP_AI_1 -- SL_AI_1,91.17,630.55,99.65,0.80,0.69,0.81,30,32,32,630.55
145,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 5 -- ROC_1,TP_AI_1 -- SL_AI_1,91.17,630.55,99.65,0.80,0.69,0.81,30,32,32,630.55
159,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 10 -- ROC_1,TP_AI_1 -- SL_AI_1,91.17,630.55,99.65,0.80,0.69,0.81,30,32,32,630.55
115,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- STOCH_30,TP_AI_1 -- SL_AI_1,106.75,604.12,106.95,0.81,0.60,0.74,36,42,42,604.12
129,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 4 -- STOCH_30,TP_AI_1 -- SL_AI_1,106.75,604.12,106.95,0.81,0.60,0.74,36,42,42,604.12
143,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 5 -- STOCH_30,TP_AI_1 -- SL_AI_1,106.75,604.12,106.95,0.81,0.60,0.74,36,42,42,604.12
157,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 10 -- STOCH_30,TP_AI_1 -- SL_AI_1,106.75,604.12,106.95,0.81,0.60,0.74,36,42,42,604.12
291,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- AVG_VOL_P_GAIN_1_2,TP_AI_2 -- SL_AI_1,0.00,374.89,0.00,0.31,0.53,0.29,138,179,179,374.89
114,BTCUSDT -- 4H -- amazon/chronos-t5-large,1 -- 3 -- STOCH_20,TP_AI_1 -- SL_AI_1,101.66,372.10,106.57,0.80,0.57,0.72,76,90,90,372.10


In [ ]:
FILE	                                    STRATEGY	                        TP_SL	                END_BALANCE__1	END_BALANCE__2	END_BALANCE__3
XRPUSDT -- 4H -- amazon/chronos-t5-small	1  --  5  --  STOCH_40	            TP_AI_2  --  SL_ATR_2	181.85	        36148.37	    313.47
XRPUSDT -- 4H -- amazon/chronos-t5-large	0.75  --  4  --  ROC_1	            TP_AI_2  --  SL_ATR_2	186.29	        16972.36	    72.13
XRPUSDT -- 4H -- amazon/chronos-t5-small	0.5  --  5  --  STOCH_40	        TP_AI_2  --  SL_ATR_2	198.27	        9382.13	        269.05
DOGEUSDT -- 4H -- amazon/chronos-t5-small	0.5  --  4  --  AVG_VOL_P_GAIN_2_1	TP_AI_2  --  SL_ATR_2	106.44	        6030.24	        73.09
ADAUSDT -- 4H -- amazon/chronos-t5-small	1  --  5  --  ROC_3	                TP_AI_2  --  SL_ATR_2	747.96	        0	            345.61
ADAUSDT -- 4H -- amazon/chronos-t5-small	1  --  3  --  ROC_3	                TP_AI_2  --  SL_ATR_2	340.08	        0	            416.36
BTCUSDT -- 1D -- amazon/chronos-t5-small	1  --  4  --  STOCH_10	            TP_AI_1  --  SL_ATR_1	1113201.9	    1.25	        1361874.11
XRPUSDT -- 1D -- amazon/chronos-t5-small	1  --  3  --  STOCH_40	            TP_AI_1  --  SL_ATR_1	103.61	        22240.09	    118.06
XRPUSDT -- 1D -- amazon/chronos-t5-large	0.75  --  10  --  STOCH_40	        TP_AI_1  --  SL_ATR_1	168.86	        9972.91	        183.04
XRPUSDT -- 1D -- amazon/chronos-t5-small	1  --  10  --  STOCH_40	            TP_AI_1  --  SL_ATR_1	66.22	        19593.83	    119.92
ADAUSDT -- 1D -- amazon/chronos-t5-large	0.75  --  3  --  AVG_VOL_P_GAIN_3_2	TP_AI_2  --  SL_ATR_2	112.95	        23017.27	    106.22
ADAUSDT -- 1D -- amazon/chronos-t5-small	0.5  --  10  --  ROC_3	            TP_AI_1  --  SL_ATR_1	186.49	        20431.75	    166.61
XRPUSDT -- 1D -- amazon/chronos-t5-small	1  --  3  --  AVG_VOL_P_GAIN_3_2	TP_AI_1  --  SL_AI_1	458.76	        6143.56	        327.59
XRPUSDT -- 1D -- amazon/chronos-t5-small	1  --  4  --  AVG_VOL_P_GAIN_3_2	TP_AI_1  --  SL_ATR_1	251.56	        3892.85	        167.81
XRPUSDT -- 1D -- amazon/chronos-t5-small	0.5  --  10  --  ROC_3	            TP_AI_2  --  SL_ATR_2	318.4	        2260.71	        282.34
XRPUSDT -- 1D -- amazon/chronos-t5-small	0.5  --  3  --  AVG_VOL_P_GAIN_3_2	TP_AI_1  --  SL_AI_1	366.53	        811.74	        203.78
ADAUSDT -- 4H -- amazon/chronos-t5-large	1  --  3  --  AVG_VOL_P_GAIN_2_2	TP_AI_1  --  SL_ATR_1	77.42	        913.59	        189.15	
ADAUSDT -- 4H -- amazon/chronos-t5-large	1  --  4  --  AVG_VOL_P_GAIN_2_2	TP_AI_1  --  SL_AI_1	31.08	        879.02	        47.83	



In [ ]:

#################################################################################################################
###############################################  PLOTTING EXAMPLE ###############################################
#################################################################################################################



import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline



TRAIN_RATIO                     = 0.99
COIN                            = 'BTCUSDT'
PATH_USE                        = r'/Users/westhomas/Desktop/ENKI/ENKI_AI_DEVELOPMENT/_DATA_/'
FILE                            = f"{COIN} -- 1D.csv"


TRAINING_DATA                   = pd.read_csv(PATH_USE + FILE) 
TRAINING_DATA["DATE"]           = pd.to_datetime(TRAINING_DATA["DATE"], dayfirst=True)
TRAINING_DATA                   = TRAINING_DATA[['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]

TRAIN_SIZE                      = int(len(TRAINING_DATA) * TRAIN_RATIO)
TRAIN_DF                        = TRAINING_DATA.iloc[:TRAIN_SIZE].reset_index(drop=True) 
VALID_DF                        = TRAINING_DATA.iloc[TRAIN_SIZE:].reset_index(drop=True)

PIPELINE                        = ChronosPipeline.from_pretrained("amazon/chronos-t5-small", device_map="cpu", torch_dtype=torch.float32)


CONTEXT                         = torch.tensor(TRAIN_DF["CLOSE"])
PREDICTION_LENGTH               = 16
FORECAST                        = pipeline.predict(CONTEXT, PREDICTION_LENGTH) 

FORECAST_INDEX                  = range(len(TRAIN_DF), len(TRAIN_DF) + PREDICTION_LENGTH)
LOW, MEDIAN, HIGH               = np.quantile(FORECAST[0].numpy(), [0.1, 0.5, 0.9], axis=0)
VALID_DF["PREDICTIONS"]         = MEDIAN



plt.figure(figsize=(30, 4))
plt.plot(TRAIN_DF["CLOSE"], color="royalblue", label="historical data")
plt.plot(FORECAST_INDEX, MEDIAN, color="tomato", label="median forecast")
plt.fill_between(FORECAST_INDEX, LOW, HIGH, color="tomato", alpha=0.3, label="80% prediction interval")
plt.legend()
plt.grid()
plt.show()


VALID_DF



In [ ]:



#################################################################################################################
###############################################    RAW EXAMPLE    ###############################################
#################################################################################################################


TRAIN_RATIO                     = 0.97
COIN                            = 'BTCUSDT'
PATH_USE                        = r'/Users/westhomas/Desktop/ENKI/ENKI_AI_DEVELOPMENT/_DATA_/'
FILE                            = f"{COIN} -- 1D.csv"


TRAINING_DATA                   = pd.read_csv(PATH_USE + FILE) 
TRAINING_DATA["DATE"]           = pd.to_datetime(TRAINING_DATA["DATE"], dayfirst=True)


TRAIN_SIZE                      = int(len(TRAINING_DATA) * TRAIN_RATIO)
TRAIN_DF                        = TRAINING_DATA.iloc[:TRAIN_SIZE].reset_index(drop=True) 
VALID_DF                        = TRAINING_DATA.iloc[TRAIN_SIZE:].reset_index(drop=True) 

DATA_LABELS                     = [f'{COIN}_TRAINING_DATA.json', f'{COIN}_VALIDATIO_DATA.json']
DATA_LIST                       = [TRAIN_DF, VALID_DF]


def EXPORT_MODEL_TRAIN_DATA(INPUT_DF, FILENAME):
    #FEATURES_LIST = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'EMA_4', 'EMA_8', 'EMA_12', 'EMA_18', 'EMA_24', 'EMA_36', 'EMA_50', 'EMA_100', 'EMA_200']
    
    #INPUT_DF["PAST_VALUE_TEXT"]     = INPUT_DF[FEATURES_LIST].apply(lambda row: " | ".join([f"{feature}: {value:.3f}" for feature, value in zip(FEATURES_LIST, row)]), axis=1)
    #INPUT_DF["TARGET_VALUE_TEXT"]   = INPUT_DF[['TARGET']].apply(lambda row: " | ".join([f"TARGET : {value:.3f}" for feature, value in zip(FEATURES_LIST, row)]), axis=1)

    #CHRONOS_DATA                    = { "START"         : str(INPUT_DF["DATE"].iloc[0]),            # First timestamp
    #                                    "PAST_VALUES"   : INPUT_DF["PAST_VALUE_TEXT"].tolist(),     # Text-based features
    #                                    "TARGET"        : INPUT_DF["TARGET_VALUE_TEXT"].tolist()}   # Convert targets to strings
                                    



    EXCLUSION                       = ['DATE', 'TARGET']
    FEATURES                        = INPUT_DF.columns.tolist()
    FEATURES_LIST                   = [ITEM for ITEM in FEATURES if ITEM not in EXCLUSION]

    CHRONOS_DATA                    = { "START"         : str(INPUT_DF["DATE"].iloc[0]),            # First timestamp
                                        "PAST_VALUES"   : INPUT_DF[FEATURES_LIST].values.tolist(),  # Input features
                                        "TARGET"        : INPUT_DF["TARGET"].tolist()}              # Convert targets to strings


    PATH_USE                        = r'/Users/westhomas/Desktop/ENKI/ENKI_AI_DEVELOPMENT/_DATA_/'
    FULL_JSON_PATH                  = os.path.join(PATH_USE, FILENAME)
    with open(FULL_JSON_PATH, "w") as f: json.dump([CHRONOS_DATA], f)

    return CHRONOS_DATA



for A in range(len(DATA_LABELS)): OUTPUT_DATA = EXPORT_MODEL_TRAIN_DATA(DATA_LIST[A], DATA_LABELS[A])











from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer, T5Tokenizer
from datasets import load_dataset


MODEL_NAME                          = "google/t5-v1_1-small"
TOKENIZER                           = T5Tokenizer.from_pretrained(MODEL_NAME)



def PREPROCESS_FUNCTION(examples):
    inputs = TOKENIZER([str(x) for x in examples["PAST_VALUES"]], padding="max_length", truncation=True, max_length=1280)
    labels = TOKENIZER([str(x) for x in examples["TARGET"]], padding="max_length", truncation=True, max_length=1280)
    inputs["labels"] = labels["input_ids"]
    return inputs





DATASET                 = load_dataset("json", data_files={"train": os.path.join(PATH_USE, DATA_LABELS[0]), "validation": os.path.join(PATH_USE, DATA_LABELS[1])})
TOKENIZED_TRAIN_DATASET = DATASET["train"].map(PREPROCESS_FUNCTION, batched=True)
TOKENIZED_EVAL_DATASET  = DATASET["validation"].map(PREPROCESS_FUNCTION, batched=True)










MODEL = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
MODEL.to(DEVICE)

TRAINING_ARGS = TrainingArguments(
    output_dir                      = "./chronos_model",
    per_device_train_batch_size     = 16,
    num_train_epochs                = 10,
    learning_rate                   = 1e-5,
    weight_decay                    = 0.01,
    save_strategy                   = "epoch",
    logging_dir                     = "./logs",
    logging_steps                   = 10,
    evaluation_strategy             = "epoch", 
    save_total_limit                = 3,
    load_best_model_at_end          = True,  
    report_to                       = "none",
)


TRAINER = Trainer(
    model                           = MODEL,
    args                            = TRAINING_ARGS,
    train_dataset                   = TOKENIZED_TRAIN_DATASET,
    eval_dataset                    = TOKENIZED_EVAL_DATASET)


TRAINER.train()






